In [1]:
import csv
import json
import requests
import openpyxl
import pandas as pd

In [2]:
# Loading excel file
wb = openpyxl.load_workbook("xlsx_painel_full.xlsx", read_only=True)
pages = wb.sheetnames

In [3]:
#Scraping data and generating  with pandas
# for i in range(2):
#     ws = wb[pages[i]]
#     result = [list(row) for row in ws.iter_rows(values_only=True)]
#     df = pd.DataFrame(result)
#     display(df)

In [4]:
# Getting cities 
with open("cod_mun_MG.csv", mode="r", encoding="utf-8") as fobj:
    reader = csv.reader(fobj)
    mun_cod_dict = { str(row[1])[:6]:row[0] for row in reader}
# for (key, val) in mun_cod_dict.items():
#     print(key, val)

In [5]:
def excel_data(page):
    ws = wb[pages[page]]
    data = [list(row) for row in ws.iter_rows(values_only=True)]
    headers = [data[0][3], data[0][1]] 
    values = [[row[3], row[1]] for row in data[1:]]
    
    result = []
    for value in values:
        result.append(dict(zip(headers, value)))
    return result

In [7]:
# Scraping data and generating csv
confirmed = excel_data(0)
deaths = excel_data(1)
result = []
for key, val in mun_cod_dict.items():
    result.append([val, 
            sum([item["NUM_CASOS"] for item in confirmed if item["CodigoIBGE"] == str(key)]),
            sum([item["NUM_OBITOS"] for item in deaths if item["CodigoIBGE"] == str(key)])
        ])
# Generating csv
output_file = "test.csv"
with open(output_file, mode="w", encoding="utf-8", newline="") as fobj:
    writer = csv.DictWriter(fobj, fieldnames=["municipio", "confirmados", "mortes"])
    writer.writeheader()
    confirmed = 0
    death = 0
    for municipio in result:
        confirmed += int(municipio[1])
        death += int(municipio[2])
        writer.writerow(
            {
                "municipio": municipio[0],
                "confirmados": municipio[1],
                "mortes": municipio[2],
            }
        )
    writer.writerow(
            {
                    "municipio": "TOTAL NO ESTADO",
                    "confirmados": confirmed,
                    "mortes": death,
            }
    )
    writer.writerow(
            {
                    "municipio": "Importados/Indefinidos",
                    "confirmados": "",
                    "mortes": "",
            }
    )

, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 4, 0, 0, 0, 1, 0, 0, 0, 7, 0, 2, 3, 2, 2, 0, 8, 5, 3, 3, 1, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 3, 0, 3, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 0, 2, 0, 6, 0, 0, 1, 0, 2, 5, 0, 0, 7, 0, 0, 1, 0, 2, 0, 0, 1, 0, 0, 4, 0, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[1, 4, 0, 0, 0, 1, 0, 0, 0, 7, 0, 2, 3, 2, 2, 0, 8, 5, 3, 3, 1, 2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 